## Imports

In [2]:
import ed25519
import x25519
from aes import aes
import aes
import base64
import socket
import os
import time

In [12]:
data = bytes(0xfeebdaed)

print(data)

TypeError: string argument without an encoding

## Full apllication

In [3]:
# const
address = '192.168.41.121'    
port = 20582        

# ed25519 private Key
priv_ed25519 = ed25519.keys.SigningKey(base64.b64decode("06DAozc2xb72sdoseInPHTLwhg+E3+pC+qPhxCh12To="))
print(f"Public Key to save in arduino :{base64.b64encode(priv_ed25519.get_verifying_key().to_bytes())}")

Public Key to save in arduino :b'GskUXMWVAz26Ko+AAywcB75HGZurjfnznrgsxaSY1mc='


In [290]:

# TCP server
s = socket.socket()           
s.bind((address, port))         
print ("socket binded to %s" %(port))
s.listen(5)    
print ("socket is listening")    

# connect first client
c, addr = s.accept()    
print (f"Got connection from {addr} \n" )
c.settimeout(5)

# receive public ed25519 key from server then accept
# Normally fisrt time add to known host
client_pub_ed25519 = ed25519.keys.VerifyingKey(base64.b64decode(c.recv(44).decode("ascii"))) 
c.send('accept'.encode("ascii"))
print(f"client with \n{client_pub_ed25519.to_bytes().hex()}\npublic key is accepted\n")

# get public x25519 from client and control with signature
# control signature will be usefull if public ed25519 can be verified with known host
base64_client_pub_x25519 = c.recv(44).decode("ascii")
client_pub_x25519 = base64.b64decode(base64_client_pub_x25519)
client_signature = c.recv(64)

# control signature 
try:
    client_pub_ed25519.verify(client_signature.hex(), base64_client_pub_x25519.encode("ascii"), encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")

# authenticate us
priv_x25519 =  os.urandom(32)
pub_x25519 = x25519.scalar_base_mult(priv_x25519)
encoded_pub_x25519 = base64.b64encode(pub_x25519)

signature = priv_ed25519.sign(encoded_pub_x25519)
c.send(encoded_pub_x25519)
c.send(signature)

shared_from_client = c.recv(32)
print(f"\nShared from client  : {(shared_from_client).hex()}")

calculate_shared = x25519.curve25519(priv_x25519, client_pub_x25519)

print(f"Shared              : {(calculate_shared).hex()}")
symkey =  aes.core.key_expansion(aes.utils.int2arr8bit(int.from_bytes(calculate_shared,'big'),32),256 )


# c.send("\nCan you encrypt yours messages please it's nice".encode("ascii"))

# time.sleep(1)
# encrypted += c.recv(2048)
s.close()

socket binded to 20582
socket is listening
Got connection from ('192.168.41.97', 58826) 

client with 
0b33207d501e5b99786d612da9f624388796843687089d64023c38e8c13f5784
public key is accepted

The signature is valid.

Shared from client  : c910704b3ed4aa9ed21077c8a4637a0ca066d665e587c30f869e1150974e3e4f
Shared              : c910704b3ed4aa9ed21077c8a4637a0ca066d665e587c30f869e1150974e3e4f


Decrypt last message with shared key from agreement

In [295]:
encrypted = c.recv(2048)

In [294]:
dlast_ciphertext = [0]*16

In [296]:
print(encrypted.hex())
print("nb char", len(encrypted))
msg = ""
# print(aes.utils.int2arr8bit(int.from_bytes(encrypted, 'big'), len(encrypted)))
try:
    for x in range(int(len(encrypted)/16)):
        ciphertext = aes.utils.int2arr8bit(int.from_bytes(encrypted[x*16:x*16+16:], 'big'), 16)
        plaintext = list(a^b for a,b in zip(aes.core.decryption(ciphertext, symkey),dlast_ciphertext))
        dlast_ciphertext = ciphertext
        msg += aes.utils.arr8bit2int(plaintext).to_bytes(len(plaintext),'big').decode('ascii')
except:
    print(f"error with cipher text {aes.utils.arr8bit2int(plaintext).to_bytes(len(plaintext),'big').hex()}")
print(msg)


4e7b955c06fd40fd2fea59e4d476e6d0e9210de77fff683143ff6603888cab55838257ac46d813369ea324bd08107d0073699e70e1758d85f12a7e2b559314b432aefe51ddcb26e177d6ed2e9416da2fefa6d6897840b57f25324d8f891066902b13567595719e069aad8878f3e14b19
nb char 112
i'm happy to see it ! try more it's asciiencryted work i try to reijbsojbhsfd asfaassaasasasasasasaass          


Encrypt

In [4]:
elast_ciphertext = [0]*16

In [5]:
calculate_shared = base64.b64decode("yypXgOK+Ehk1LieEHIhJmek4yJAsd2h5ktDfBy7HMWg=")
symkey = aes.core.key_expansion(aes.utils.int2arr8bit(int.from_bytes(calculate_shared,'big'),32),256 )
text = "Data\0\0\0\0\0\0\0\0\0\0\0\x01\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\x01\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\x02"
print(text)
# text += '\0'*(16-len(text)%16)
print("nb char", len(text))
encryptedmsg = bytes()

for x in range(int(len(text)/16)):
    plaintext = aes.utils.int2arr8bit(int.from_bytes(text[x*16:x*16+16:].encode("ascii"), 'big'), 16)
    ciphertext = aes.core.encryption(list(a^b for a,b in zip(plaintext,elast_ciphertext)), symkey)
    elast_ciphertext = ciphertext
    encryptedmsg += aes.utils.arr8bit2int(ciphertext).to_bytes(len(ciphertext),'big')
print(encryptedmsg)

Data                                         
nb char 48
b"P\x0b\x97\xee\xf7\xca<^\x9c\xba\x94C0\xa3\x987\xca'*\xaeI\xbe$\xa0\x96\xa4\x05\xfcD\x85\xf5|\xb3BE\xdf6W\xf6\xb2\x8cJ\xda*&\x98\xb1\xc7"


In [293]:
c.send(encryptedmsg)

112

## Individual cells for testing

### Signatures tests

In [9]:
privKey = ed25519.keys.SigningKey(b'0FDE8A96B2066E34FBEAC155FC5104A8C05A96C501A001300EE99F9EB261D82D',encoding='hex')
pubKey = ed25519.keys.VerifyingKey(b'7E4A530129FBE298F9E431356F0D2F5687FC21A1241064EA224687DE8BE5A24A',encoding='hex')

print("Private key (32 bytes):", privKey.to_seed().hex())
print("Public key (32 bytes): ", pubKey.to_bytes().hex())


msg = "test de signature".encode("ascii")

# signature = privKey.sign(msg, encoding='hex')
signature = b'B785B9A12C6389EDF447D470D2B38C278E5ADBC31F67FA79741B9AC8CA78FB9EB96585DE2DFD2ABCF4D769163F07AEBC412424247824EE9BF2F087FA7F5FD10B'

print("Signature (64 bytes):", signature)
try:
    pubKey.verify(signature, msg, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")


Private key (32 bytes): 0fde8a96b2066e34fbeac155fc5104a8c05a96c501a001300ee99f9eb261d82d
Public key (32 bytes):  7e4a530129fbe298f9e431356f0d2f5687fc21a1241064ea224687de8be5a24a
Signature (64 bytes): b'B785B9A12C6389EDF447D470D2B38C278E5ADBC31F67FA79741B9AC8CA78FB9EB96585DE2DFD2ABCF4D769163F07AEBC412424247824EE9BF2F087FA7F5FD10B'
The signature is valid.


In [501]:
pubKey = ed25519.keys.VerifyingKey(base64.b64decode("vRy71JXxwpbl3ZfCJCeROrUrYbxD8DMuu0qsCTIa1Oc=").hex(),encoding='hex')

msg = "hYSGOw1znznl5NO3OnI85/1kiC75dUEH0u1BZLDFLWw=".encode("ascii")

signature = b'0f2f532b651886ad13022057cf3c2acc7daf573ed48487484d62d5s4c972354de7390ff041561dfbe35877c0575a199ade4200d2ddf5efb92e9f57ad12b542102'

try:
    pubKey.verify(signature, msg, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")

Invalid signature!


### Create random keys 

In [90]:
# privKey2, pubKey2 = ed25519.keys.create_keypair()
privKey2 = ed25519.keys.SigningKey(base64.b64decode("06DAozc2xb72sdoseInPHTLwhg+E3+pC+qPhxCh12To="))
pubKey2 = privKey2.get_verifying_key()
print("Private key (32 bytes):", privKey2.to_seed().hex())
print("Public key (32 bytes): ", pubKey2.to_bytes().hex())

Private key (32 bytes): d3a0c0a33736c5bef6b1da2c7889cf1d32f0860f84dfea42faa3e1c42875d93a
Public key (32 bytes):  1ac9145cc595033dba2a8f80032c1c07be47199bab8df9f39eb82cc5a498d667


### Test Curve25519

In [107]:
print("arduino Private key (32 bytes):", privKey.to_seed().hex())
print("arduino Public key (32 bytes): ", pubKey.to_bytes().hex())

print("python Private key (32 bytes):", privKey2.to_seed().hex())
print("python Public key (32 bytes): ", pubKey2.to_bytes().hex())

pubx = x25519.scalar_base_mult(privKey.to_seed())
print("arduino Public key x(32 bytes): ",pubx.hex())
pubx2 = x25519.scalar_base_mult(privKey2.to_seed())
print("python Public key x(32 bytes): ", pubx2.hex())

shared = x25519.curve25519(privKey.to_seed(), pubx2) 
sharedPy = x25519.curve25519(privKey2.to_seed(), pubx) 

print (f"\n\nShared (arduino):\t{(shared).hex()}")
print (f"Shared (python): \t{(sharedPy).hex()}")

arduino Private key (32 bytes): d3a0c0a33736c5bef6b1da2c7889cf1d32f0860f84dfea42faa3e1c42875d93a
arduino Public key (32 bytes):  bd1cbbd495f1c296e5dd97c22427913ab52b61bc43f0332ebb4aac09321ad4e7
python Private key (32 bytes): d3a0c0a33736c5bef6b1da2c7889cf1d32f0860f84dfea42faa3e1c42875d93a
python Public key (32 bytes):  1ac9145cc595033dba2a8f80032c1c07be47199bab8df9f39eb82cc5a498d667
arduino Public key x(32 bytes):  a3f86021c755faf9bcebbca83814ab8c8b95a2e76994645ec0e3da338dc8d806
python Public key x(32 bytes):  a3f86021c755faf9bcebbca83814ab8c8b95a2e76994645ec0e3da338dc8d806


Shared (arduino):	89042f153af5b27a6cdc290749c20fd5af3b7ebaf480542f03165fbd55efca45
Shared (python): 	89042f153af5b27a6cdc290749c20fd5af3b7ebaf480542f03165fbd55efca45


### Save sever public key in arduino (Base64)

In [61]:
print(f"Public Key to save in arduino :{base64.b64encode(pubKey2.to_bytes())}")

Public Key to save in arduino :b'06DAozc2xb72sdoseInPHTLwhg+E3+pC+qPhxCh12To='
Public Key to save in arduino :b'GskUXMWVAz26Ko+AAywcB75HGZurjfnznrgsxaSY1mc='


In [91]:
privx2 =  os.urandom(32)
pubx2 = x25519.scalar_base_mult(privx2)

encoded = base64.b64encode(pubx2)

print(f"Key in hex  :{pubx2.hex()}")
print(f"Key to send :{encoded}")
 
signatureofencoded = privKey2.sign(encoded)
print(f"Signature :{signatureofencoded.hex()}")

Key in hex  :20f18bd2680f7dc245d4094ed90e823cae997f69bc828f97ee1d8f1a2eb0ed25
Key to send :b'IPGL0mgPfcJF1AlO2Q6CPK6Zf2m8go+X7h2PGi6w7SU='
Signature :cc029c64093e71460c57946138c82caa6a644b1da656a6e9c462d2e846fc32a8d718862f698b79c289b6d438f262a883d60266c9187101416166ca687bc7460c


In [81]:
try:
    pubKey2.verify(signatureofencoded.hex(), encoded, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")

The signature is valid.


### From public x25519 key of arduino create shared key

In [59]:
arduinoKey = base64.b64decode("WkS78GGKoSTRUB6/MOKcixwXDz9Xl0expuPZ5sES9C4=")
print(arduinoKey.hex())

realshared = x25519.curve25519(privKey2.to_seed(), arduinoKey)

print (f"Shared : \t{(realshared).hex()}")

5a44bbf0618aa124d1501ebf30e29c8b1c170f3f579747b1a6e3d9e6c112f42e
Shared : 	aff041dea1c51a458a5d3aa15027c191c9dd6f9f17f1fac97e3a720de383541c


### TCP server 

In [92]:
s = socket.socket()        
print ("Socket successfully created")

port = 20582               

s.bind(('192.168.127.121', port))        

print ("socket binded to %s" %(port))
 
s.listen(5)    
print ("socket is listening")         

Socket successfully created
socket binded to 20582
socket is listening


In [13]:
s.close()

### Accept

In [93]:
c, addr = s.accept()    
print ('Got connection from', addr )

c.settimeout(5)

Got connection from ('192.168.127.97', 54298)


In [ ]:
print (c.recv(44).decode("ascii"))
c.send('accept'.encode("ascii"))

print (c.recv(44).decode("ascii"))
print (c.recv(64).hex())

c.send(encoded)
c.send(signatureofencoded)

print (c.recv(64).hex()) 
print (c.recv(32).decode("ascii")) 
print (c.recv(32).hex()) 

#   c.close()


In [31]:
data = bytes()
try:
    print (c.recv(44).decode("ascii"))
    c.settimeout(0.01)
    while True:
        data +=c.recv(1)
except:
    print(data)

b''


### Tests AES encrypting

In [154]:
pt = 0x00112233445566778899aabbccddeeff

mk =  int.from_bytes(calculate_shared,'big')

cipher = aes.aes(mk, 256, mode='CTR', padding='PKCS#7')
print(mk)

ct = cipher.enc_once(pt)
print(ct)
print("0x"+hex(aes.utils.arr8bit2int(ct))[2:].zfill(32))

pr = cipher.dec_once(ct)
print(pr)
print("0x"+hex(aes.utils.arr8bit2int(pr))[2:].zfill(32))

print(cipher.iv) 

44491891957684176482331504626869073669219334627995553290301045895970595504980
[99, 210, 123, 31, 17, 165, 233, 22, 177, 118, 35, 254, 195, 101, 187, 43]
0x63d27b1f11a5e916b17623fec365bb2b
[0, 17, 34, 51, 68, 85, 102, 119, 136, 153, 170, 187, 204, 221, 238, 255]
0x00112233445566778899aabbccddeeff
[91, 34, 218, 213, 116, 251, 217, 246, 71, 36, 1, 163, 191, 29, 28, 20]


c:\Users\Isaac\anaconda3\envs\HeArc\lib\site-packages\aes\utils\_check_tools.py:59: UserWarning: Initail Vector is randomly selected: [91, 34, 218, 213, 116, 251, 217, 246, 71, 36, 1, 163, 191, 29, 28, 20]
  warnings.warn("Initail Vector is randomly selected: " + str(iv))
